In [7]:
import os
from typing import Any

from dotenv import load_dotenv
from utils.agent_visualizer import print_activity

from claude_agent_sdk import ClaudeAgentOptions, ClaudeSDKClient, ResultMessage

# 02 - The Observability Agent

In the previous notebooks we have built a basic research agent and a Chief of Staff multi-agent framework. While the agents we have built are already powerful, they were still limited in what they could do: the web search agent is limited to searching the internet and our Chief of Staff agent was limited to interacting with its own filesystem.

This is a serious constraint: real-world agents often need to interact with other systems like databases, APIs, file systems, and other specialized services. [MCP (Model Context Protocol)](https://modelcontextprotocol.io/docs/getting-started/intro) is an open-source standard for AI-tool integrations that allows for an easy connection between our agents and these external systems. In this notebook, we will explore how to connect MCP servers to our agent.

**Need more details on MCP?** For comprehensive setup instructions, configuration best practices, and troubleshooting tips, see the [Claude Code MCP documentation](https://docs.claude.com/en/docs/claude-code/mcp).

## Introduction to the MCP Server
### 1. The Git MCP server

Let's first give our agent the ability to understand and work with Git repositories. By adding the [Git MCP server](https://github.com/modelcontextprotocol/servers/tree/main/src/git) to our agent, it gains access to 13 Git-specific tools that let it examine commit history, check file changes, create branches, and even make commits. This transforms our agent from a passive observer into an active participant in your development workflow. In this example, we'll configure the agent to explore a repository's history using only Git tools. This is pretty simple, but knowing this, it is not difficult to imagine agents that can automatically create pull requests, analyze code evolution patterns, or help manage complex Git workflows across multiple repositories.

In [ ]:
# define our git MCP server (it was downloaded when you ran uv sync as it is defined in the pyproject.toml file)
git_mcp: dict[str, Any] = {
    "git": {
        "command": "uv",
        "args": ["run", "python", "-m", "mcp_server_git", "--repository", os.getcwd()],
    }
}

In [4]:
messages = []
async with (
    ClaudeSDKClient(
        options=ClaudeAgentOptions(
            model="claude-sonnet-4-5",
            mcp_servers=git_mcp,
            allowed_tools=[
                "mcp__git"
            ],  # For MCP tools, in allowed tools we must add the mcp__serverName__toolName format or mcp__serverName to enable all
            permission_mode="acceptEdits",  # auto-accept file edit permissions
        )
    ) as agent
):
    await agent.query(
        "Use ONLY your git mcp tools to quickly explore this repo's history and gimme a brief summary."
    )
    async for msg in agent.receive_response():
        print_activity(msg)
        messages.append(msg)

🤖 Thinking...
🤖 Using: mcp__git()
✓ Tool completed
🤖 Thinking...
🤖 Using: Bash()
🤖 Using: Bash()
🤖 Using: Bash()
✓ Tool completed
✓ Tool completed
✓ Tool completed
🤖 Thinking...


In [ ]:
from claude_agent_sdk import ResultMessage

result = next((m.result for m in reversed(messages) if isinstance(m, ResultMessage)), None)
print(f"\nResult:\n{result}")

### 2. The GitHub MCP server

Now let's level up from local Git operations to full GitHub platform integration. By switching to the [official GitHub MCP server](https://github.com/github/github-mcp-server/tree/main), our agent gains access to over 100 tools that interact with GitHub's entire ecosystem – from managing issues and pull requests to monitoring CI/CD workflows and analyzing code security alerts. This server can work with both public and private repositories, giving your agent the ability to automate complex GitHub workflows that would typically require multiple manual steps.

#### Step 1: Set up your GitHub Token

You need a GitHub Personal Access Token. Get one [here](https://github.com/settings/personal-access-tokens/new) and put in the .env file as ```GITHUB_TOKEN="<token>"```
> Note: When getting your token, select "Fine-grained" token with the default options (i.e., public repos, no account permissions), that'll be the easiest way to get this demo working.

Also, for this example you will have to have [Docker](https://www.docker.com/products/docker-desktop/) running on your machine. Docker is required because the GitHub MCP server runs in a containerized environment for security and isolation.

**Docker Quick Setup:**
- Install Docker Desktop from [docker.com](https://www.docker.com/products/docker-desktop/)
- Ensure Docker is running (you'll see the Docker icon in your system tray)
- Verify with `docker --version` in your terminal
- **Troubleshooting:** If Docker won't start, check that virtualization is enabled in your BIOS. For detailed setup instructions, see the [Docker documentation](https://docs.docker.com/get-docker/)

#### Step 2: Define the mcp server and start the agent loop!

In [6]:
# define our github mcp server
load_dotenv(override=True)
github_mcp: dict[str, Any] = {
    "github": {
        "command": "docker",
        "args": [
            "run",
            "-i",
            "--rm",
            "-e",
            "GITHUB_PERSONAL_ACCESS_TOKEN",
            "ghcr.io/github/github-mcp-server@sha256:26c7b1b8ac6e3e85f02a5dbd3b2061686d1fd2f6c8191ef29a2ab548f2c5404f",
        ],
        "env": {"GITHUB_PERSONAL_ACCESS_TOKEN": os.environ.get("GITHUB_TOKEN")},
    }
}

In [7]:
# run our agent
messages = []
async with ClaudeSDKClient(
    options=ClaudeAgentOptions(
        model="claude-sonnet-4-5",
        mcp_servers=github_mcp,
        allowed_tools=["mcp__github"],
        permission_mode="acceptEdits",  # auto-accept permissions
    )
) as agent:
    await agent.query(
        "Use ONLY your GitHub MCP tools to search for the anthropics/claude-agent-sdk-python repository and and give me a couple facts about it"
    )
    async for msg in agent.receive_response():
        print_activity(msg)
        messages.append(msg)

🤖 Thinking...
🤖 Using: mcp__github__search_repositories()
✓ Tool completed
🤖 Thinking...
🤖 Using: mcp__github__get_file_contents()
✓ Tool completed
🤖 Thinking...


In [ ]:
from claude_agent_sdk import ResultMessage

result = next((m.result for m in reversed(messages) if isinstance(m, ResultMessage)), None)
print(f"\nResult:\n{result}")

### 3. Custom Tools with SDK MCP Servers

**What**: SDK MCP Servers let you create custom tools directly in Python using the `@tool` decorator and `create_sdk_mcp_server()`. Unlike external MCP servers (Git, GitHub) that run as separate processes, SDK MCP servers run in-process—same Python runtime as your agent.

**Why**: External MCP servers (like Git and GitHub above) are great for integrating existing systems, but they have overhead: subprocess communication, Docker containers, network calls. SDK MCP servers give you:
- **Zero subprocess overhead**: Runs in your Python process
- **Direct state access**: Can access your application's data, databases, and objects
- **Simpler deployment**: No external dependencies or Docker
- **Type safety**: Full Python type hints and IDE support
- **Custom logic**: Any Python code you can write becomes a tool

**When to use**:
- **External MCP**: Existing systems (GitHub API, databases, services you don't control)
- **SDK MCP**: Custom logic, internal APIs, data transformations, calculations you control

**How**: Use `@tool` decorator to define tools, then `create_sdk_mcp_server()` to bundle them. The agent accesses them via `mcp__servername__toolname` in `allowed_tools`.

Below we create a monitoring server with two custom tools:

In [9]:
from claude_agent_sdk import create_sdk_mcp_server, tool

# Define custom tools with @tool decorator
# @tool(name, description, input_schema)
# Returns dict with content array (MCP tool response format)

@tool(
    "check_service_health",
    "Check health status of a service",
    {"service_name": str}  # Input schema: simple type mapping
)
async def check_service_health(args: dict) -> dict:
    # In production, this would call real health endpoints
    # For demo, we use mock data
    services = {
        "api": {"status": "healthy", "latency_ms": 45},
        "database": {"status": "healthy", "latency_ms": 12},
        "cache": {"status": "degraded", "latency_ms": 203}
    }
    
    service = args.get("service_name", "").lower()
    if service in services:
        info = services[service]
        # Return MCP tool response format
        return {
            "content": [{
                "type": "text",
                "text": f"{service}: {info['status']} (latency: {info['latency_ms']}ms)"
            }]
        }
    return {
        "content": [{
            "type": "text",
            "text": f"Service '{service}' not found"
        }]
    }

@tool(
    "calculate_error_rate",
    "Calculate error rate percentage",
    {"errors": int, "total_requests": int}  # Multiple parameters with types
)
async def calculate_error_rate(args: dict) -> dict:
    errors = args["errors"]
    total = args["total_requests"]
    rate = (errors / total * 100) if total > 0 else 0
    
    # Add business logic - determine severity
    severity = "critical" if rate > 5 else "warning" if rate > 1 else "normal"
    
    return {
        "content": [{
            "type": "text",
            "text": f"Error rate: {rate:.2f}% ({errors}/{total}) - Severity: {severity}"
        }]
    }

# Bundle tools into an SDK MCP server
# This creates an in-process MCP server (no subprocess\!)
monitoring_server = create_sdk_mcp_server(
    name="monitoring",  # Server name
    version="1.0.0",
    tools=[check_service_health, calculate_error_rate]  # List of @tool decorated functions
)

# Now monitoring_server can be passed to mcp_servers option
# Tools become: mcp__monitoring__check_service_health
#               mcp__monitoring__calculate_error_rate

In [10]:
# Use SDK MCP server in agent
messages = []
async with ClaudeSDKClient(
    options=ClaudeAgentOptions(
        model="claude-sonnet-4-5",
        mcp_servers={"monitor": monitoring_server},
        allowed_tools=["mcp__monitor"]  # Enable all monitoring tools
    )
) as agent:
    await agent.query(
        "Check health of all services (api, database, cache) and calculate error rate if we had 47 errors out of 1200 requests"
    )
    async for msg in agent.receive_response():
        print_activity(msg)
        messages.append(msg)
        if isinstance(msg, ResultMessage):
            print(f"Result: {msg.result}")

🤖 Thinking...
🤖 Using: mcp__monitor__check_service_health()
🤖 Using: mcp__monitor__check_service_health()
🤖 Using: mcp__monitor__check_service_health()
🤖 Using: mcp__monitor__calculate_error_rate()
✓ Tool completed
✓ Tool completed
✓ Tool completed
✓ Tool completed
🤖 Thinking...
Result: ## Service Health Status

- **api**: healthy (latency: 45ms)
- **database**: healthy (latency: 12ms)
- **cache**: degraded (latency: 203ms)

## Error Rate Analysis

**Error rate: 3.92%** (47 errors out of 1200 requests)
- Severity: warning

The cache service is showing degraded performance with elevated latency (203ms). The error rate of 3.92% has reached warning levels and may need attention.


## Real use case: An observability agent

Now, with such simple setup we can already have an agent acting as self-healing software system!

In [9]:
load_dotenv(override=True)

prompt = """Monitor the GitHub Actions workflows for facebook/react.
Look at the last triggered CI pipeline. 
1. Analyze the trigger for the pipeline
2. Identify whether the pipeline passed or not
3. If it failed, explain which test failed
4. Identify whether human involvement is required

IMPORTANT: Do not raise a PR, issue, or bug on github yet. Just give me a summary of your findings and plan.

Focus on the 'CI' workflow specifically. Use your Github MCP server tools!"""

github_mcp: dict[str, Any] = {
    "github": {
        "command": "docker",
        "args": [
            "run",
            "-i",
            "--rm",
            "-e",
            "GITHUB_PERSONAL_ACCESS_TOKEN",
            "ghcr.io/github/github-mcp-server@sha256:26c7b1b8ac6e3e85f02a5dbd3b2061686d1fd2f6c8191ef29a2ab548f2c5404f",
        ],
        "env": {"GITHUB_PERSONAL_ACCESS_TOKEN": os.environ.get("GITHUB_TOKEN")},
    }
}

messages = []
async with ClaudeSDKClient(
    options=ClaudeAgentOptions(
        model="claude-sonnet-4-5",
        mcp_servers=github_mcp,
        allowed_tools=["mcp__github"],
        permission_mode="acceptEdits",
    )
) as agent:
    await agent.query(prompt)
    async for msg in agent.receive_response():
        print_activity(msg)
        messages.append(msg)

🤖 Thinking...
🤖 Using: TodoWrite()
✓ Tool completed
🤖 Thinking...
🤖 Using: TodoWrite()
✓ Tool completed
🤖 Using: mcp__github__list_workflows()
✓ Tool completed
🤖 Thinking...
🤖 Using: mcp__github__list_workflow_runs()
✓ Tool completed
🤖 Thinking...
🤖 Using: TodoWrite()
✓ Tool completed
🤖 Thinking...
🤖 Using: mcp__github__get_workflow_run()
✓ Tool completed
🤖 Thinking...
🤖 Using: TodoWrite()
✓ Tool completed
🤖 Using: mcp__github__get_job_logs()
✓ Tool completed
🤖 Thinking...
🤖 Using: mcp__github__list_workflow_jobs()
✓ Tool completed
🤖 Thinking...
🤖 Using: WebFetch()
✓ Tool completed
🤖 Thinking...
🤖 Using: mcp__github__get_job_logs()
✓ Tool completed
🤖 Thinking...
🤖 Using: mcp__github__get_job_logs()
✓ Tool completed
🤖 Thinking...
🤖 Using: TodoWrite()
✓ Tool completed
🤖 Using: mcp__github__get_job_logs()
✓ Tool completed
🤖 Thinking...
🤖 Using: TodoWrite()
✓ Tool completed
🤖 Using: TodoWrite()
✓ Tool completed
🤖 Thinking...


In [ ]:
from claude_agent_sdk import ResultMessage

result = next((m.result for m in reversed(messages) if isinstance(m, ResultMessage)), None)
print(f"\nResult:\n{result}")

### Observability Agent as Module

The `observability_agent/agent.py` file contains the same minimal helper functions as the research agent or chief of staff agent, just enhanced for GitHub monitoring. 

As before, to use it as a module in your Python code:

In [ ]:
from observability_agent.agent import send_query

result = await send_query(
    "Check the CI status for the last 2 runs in anthropics/claude-agent-sdk-python. Just do 3 tool calls, be efficient."
)
print(f"Monitoring result: {result}")

We can do multi-turn conversations with this agent as well:

In [ ]:
# Example 2: Multi-turn conversation for deeper monitoring
result1 = await send_query("What's the current CI status for facebook/react?")
print(f"Initial check: {result1[:250]}...\n")

In [ ]:
# Continue the conversation to dig deeper
result2 = await send_query(
    "Are there any flaky tests in the recent failures? You can only make one tool call.",
    continue_conversation=True,
)
print(f"Follow-up analysis: {result2[:250]}...")

## Conclusion

We've demonstrated how the Claude Agent SDK enables powerful agent systems from basic integration to production deployment.

**What You've Learned:**

**From Notebook 00 (Research Agent)**:
- Core SDK fundamentals with `query()` and `ClaudeSDKClient`
- Basic tool usage with WebSearch and Read
- Simple agent loops and conversation management

**From Notebook 01 (Chief of Staff)**:
- Multi-agent coordination through subagents
- Governance through hooks and custom commands
- Enterprise-ready agent architectures
- Custom permissions and programmatic agents

**From Notebook 02 (Advanced Topics)**:
- **External Integration**: Git and GitHub MCP servers for system integration
- **Custom Tools**: SDK MCP servers for in-process tool creation

You now have the foundation to build production-ready agent systems that integrate with external tools, deploy reliably, handle real-world scale, and extend with reusable capabilities.

**Next Steps**:
- Explore [Claude Agent SDK documentation](https://docs.claude.com/en/api/agent-sdk)
- Browse the [Plugin Hub](https://github.com/jeremylongshore/claude-code-plugins-plus) (227+ plugins)
- Create your own skills and share with the community
- Deploy agents to production with confidence

The complete agent implementations in `research_agent/`, `chief_of_staff_agent/`, and `observability_agent/` directories provide production-ready starting points for your projects.